# Analyse de la prévalence de la maladie

## Nettoyage des données

Préambule

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

Chargement des données

In [ ]:
from getting_started import df_patient, df_pcr, pd

df_patient = df_patient.convert_dtypes()
df_pcr = df_pcr.convert_dtypes()

Nettoyage et déduplication

In [ ]:
from entity_resolution import detect_duplicates

df_patient = detect_duplicates(df_patient)

In [ ]:
df_patient.head(10)

In [ ]:
df_patient["dedup_id"] = df_patient.index.to_frame().astype("Int64")

df = df_patient[df_patient.duplicated("phone_number", keep=False)].dropna().reset_index()
df = df[["phone_number", "patient_id"]]
df = df.merge(df, on="phone_number").drop(columns="phone_number")
df = df.groupby(["patient_id_x"]).min()
df = df.rename_axis("patient_id").rename(columns={"patient_id_y": "dedup_id"}).astype("Int64")

df_patient.update(df)
df_patient = df_patient.convert_dtypes()

df_patient

In [ ]:
df_patient.info()

Taux de déduplication

In [ ]:
len(df_patient[df_patient.dedup_id.duplicated(keep=False)]) / len(df_patient)

Conversion des résultats de test PCR en variable catégorielle

In [ ]:
df_pcr.pcr = pd.Categorical(df_pcr.pcr.str[0], categories=["N", "P"], ordered=True)

df_pcr.pcr.value_counts()

Jointure avec le référentiel de patient

In [ ]:
df_pcr = df_pcr.merge(df_patient["dedup_id"], left_on="patient_id", right_index=True, validate="m:1")

df_pcr.pcr.value_counts()

Patients affectés par COVID-19

In [ ]:
df_prevalence = (
    df_pcr[["dedup_id", "pcr"]]
    .groupby("dedup_id").max()
    .rename(columns={"pcr": "affected"})
    .merge(df_patient, on="dedup_id")
    .set_index("dedup_id")
)

df_prevalence

Prévalance à l'échelle de l'âge

In [ ]:
df_prevalence["age_category"] = pd.cut(df_prevalence.age, bins=[0, 10, 20, 30, 40, 100], include_lowest=True, ordered=True)

df_prevalence[["age", "age_category"]]

In [ ]:
df_prevalence_age = (
    df_prevalence[["age_category", "affected"]]
    .replace({"affected": {"N": 0, "P": 1}})
    .groupby("age_category").agg(["sum", "count"])
    .droplevel(level=0, axis="columns")
    .rename(columns={"sum": "affected", "count": "tested"})
)

fig, ax = plt.subplots(figsize=(20, 8))
df_prevalence_age.plot.bar(stacked=True, ax=ax)

Prévalance à l'échelle de l'état

In [ ]:
df_prevalence_state = (
    df_prevalance[["state", "affected"]]
    .replace({"affected": {"N": 0, "P": 1}})
    .groupby("state").agg(["sum", "count"])
    .droplevel(level=0, axis="columns")
    .rename(columns={"sum": "affected", "count": "tested"})
)

fig, ax = plt.subplots(figsize=(20, 8))
df_prevalence_state.plot.bar(stacked=True, ax=ax)

Prévalance à l'échelle code postal

In [ ]:
df_prevalence_postcode = (
    df_prevalence[["postcode", "affected"]]
    .replace({"affected": {"N": 0, "P": 1}})
    .groupby("postcode").agg(["sum", "count"])
    .droplevel(level=0, axis="columns")
    .rename(columns={"sum": "affected", "count": "tested"})
)

Cartography

In [ ]:
from ipyleaflet import Map, basemaps

map_ = Map(basemap=basemaps.OpenStreetMap.BlackAndWhite, center=(-25.8, 136.8698), zoom=4)

In [ ]:
from ipyleaflet import Choropleth
from branca.colormap import linear
import json

geo_data = json.load(open("aus_state.geojson"))

choro_data = (
    df_prevalence_state["affected"]
    .rename(index={
        "nsw": 0,
        "vic": 1,
        "qld": 2,
        "sa": 3,
        "wa": 4,
        "tas": 5,
        "nt": 6,
        "act": 7
    })
).to_dict()

choro_data[8] = 0   # Other territories

choropleth = Choropleth(
    geo_data=geo_data,
    choro_data=choro_data,
    colormap=linear.YlOrRd_04,
    style={'fillOpacity': 0.8, 'dashArray': '5, 5'},
)

map_.add_layer(choropleth)

In [ ]:
from ipyleaflet import Heatmap

locations = (
    pd.read_csv(
        "australian_postcodes.csv",
        index_col="id",
        usecols=("id","postcode", "long", "lat"),
        dtype={"id": "int64", "postcode": "str", "long": "float", "lat": "float"}
    )
    .rename(columns={"long": "longitude", "lat": "latitude"})
    .drop_duplicates("postcode", keep="first")
    .assign(altitude=0.0)
    .set_index("postcode")
)

locations = (
    df_prevalence.merge(locations, left_on="postcode", right_index=True)
    [["latitude", "longitude", "altitude"]].values.tolist()
)

heatmap = Heatmap(locations=locations, radius=10, min_opacity=1)

map_.add_layer(heatmap)

In [ ]:
map_